# 0.0 - Imports

In [13]:
import pandas  as pd
import numpy   as np
import seaborn as sns 
import math

from scipy.stats import chi2_contingency
from statsmodels.stats import power, proportion

## 0.1 - Helper Functions

##  0.2 - Carregamento dos Dados

In [14]:
df = pd.read_csv('../data/raw/ab_data.csv').copy()
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# 1.0 - Escolha do Método

In [15]:
df1 = df.copy()

- Para solucionar esse problema, vamos utilizar o teste A/B + o Teste de Hipótese

# 2.0 - Design do Experimento

In [16]:
df2 = df1.copy()

## 2.1 - Formulação das Hipóteses

**Hipótese Nula:** A página antiga e a página nova têm a mesma taxa de conversão, isto é, ambas possuem 13%.

**Hipótese Alternativa:** A página com o design novo tem uma taxa de conversão diferente de 13%.(Teste Bicaudal.)

**Métrica:** A conversão na compra do teclado.

## 2.2 - Parâmetros da Amostra

In [17]:
# ---------------------- Determinação dos Parâmetros Para Dimensionar o Tamanho das Amostras -----------------------

# Nível de Confiança
nivel_confianca = 0.95

# Nivel de Significância
nivel_significancia = 1 - nivel_confianca

# Effect Size
# Conversões das páginas atual e nova.
conversao_pagina_atual = 0.13
conversao_pagina_nova  = 0.15

# Cálculo do effect_size
effect_size = proportion.proportion_effectsize(conversao_pagina_atual, conversao_pagina_nova)

# Poder Estatístico
power_stats = 0.8

## 2.3 - Tamanho da Amostra

In [18]:
# ------------------------------- Cálculo do Tamanho da Amostra -----------------------------------------------

# Tamanho da Amostra
tamanho_amostra = power.NormalIndPower().solve_power(effect_size=effect_size , power=power_stats, alpha=nivel_significancia)

# Reajustes
tamanho_amostra = math.ceil(tamanho_amostra)

print('O tamanho da amostra para o grupo de controle é {}.'.format(tamanho_amostra))
print('O tamanho da amostra para o grupo de tratamento é {}.'.format(tamanho_amostra))
print('O tamanho total da amostra é {}.'.format(tamanho_amostra*2))

O tamanho da amostra para o grupo de controle é 4720.
O tamanho da amostra para o grupo de tratamento é 4720.
O tamanho total da amostra é 9440.


# 3.0 - Coleta e Prepação dos Dados da Amostra

In [19]:
df3 = df2.copy()

## 3.1 - Preparação dos Dados

### 3.1.1 - Dimensão dos Dados

In [20]:
# ------ Verificação da Dimensão dos Dados --------------

print('Número de Linhas: {}.'.format(df3.shape[0]))
print('Número de Colunas: {}.'.format(df3.shape[1]))

Número de Linhas: 294478.
Número de Colunas: 5.


### 3.1.2 - Check NA

In [21]:
# -------------- Verificação de Dados Nulos -------------------
df3.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

### 3.1.3 - Tipagem dos Dados 

In [22]:
# ---------------- Verificação da Natureza dos Dados -------------------
df3.dtypes

user_id          int64
timestamp       object
group           object
landing_page    object
converted        int64
dtype: object

## 3.2 - Coleta dos Dados de Amostra

### 3.2.1 - Duplicidade das Flags

In [23]:
# Verificação para saber se existe usuários com duas flags diferentes.
df_flags = df3[['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

# renomeando a coluna user_id
df_flags = df_flags.rename(columns={'user_id':'number_users'})

# exibindo o dataframe
df_flags

,group,landing_page,number_users
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [24]:
# ----------- Remoção das Duplicidades ------------------------

# dataframe com os usuários duplicados
df_users_duplicates = df3[['user_id', 'group']].groupby('user_id').count().reset_index().query('group > 1')

# colocar os user_id em uma lista.
list_users_duplicates = df_users_duplicates['user_id'].values.tolist()

# Removendo as duplicidades do dataframe principal
df3 = df3[~df3['user_id'].isin(list_users_duplicates)].copy()

# Porcentagem removida
percent = round(len(list_users_duplicates)*2/df2.shape[0]*100, 2)
print('Porcentagem removida: {}%.'.format(percent))

# -------------------- Verificação da remoção das duplicidades --------------------

# Verificação para saber se existe usuários com duas flags diferentes.
df_flags = df3[['user_id', 'group', 'landing_page']].groupby(['group', 'landing_page']).count().reset_index()

# renomeando a coluna user_id
df_flags = df_flags.rename(columns={'user_id':'number_users'})

# exibindo o dataframe
df_flags

Porcentagem removida: 2.64%.


,group,landing_page,number_users
0,control,old_page,143293
1,treatment,new_page,143397


### 3.2.2 - Amostragem Aleatória dos Grupos de Controle e Tratamento

In [25]:
# Amostragem do Grupo de Controle
df_control_sample = df3[df3['group'] == 'control'].sample(n=tamanho_amostra, random_state=42)

# Amostragem do Grupo de Tratamento
df_treatment_sample = df3[df3['group'] == 'treatment'].sample(n=tamanho_amostra, random_state=42)

# Exibição
print('Tamando da amostra do grupo de controle: {}.'.format(df_control_sample.shape[0]))
print('Tamando da amostra do grupo de tratamento: {}.'.format(df_treatment_sample.shape[0]))

Tamando da amostra do grupo de controle: 4720.
Tamando da amostra do grupo de tratamento: 4720.


## 3.3 -  Cálculo da Conversão dos Grupos(Taxa de Conversão)

In [26]:
# ------------------------ Métrica Grupo de Controle --------------------

# Quantidade de Conversões
sales = df_control_sample[df_control_sample['converted'] == 1]['converted'].sum()

# Quantidade de Visitas
visit = len(df_control_sample)

# Taxa de Conversão do Grupo de Controle
taxa_conversao_controle = round(sales/visit, 4)

# ------------------------ Métrica Grupo de Tratamento --------------------

# Quantidade de Conversões
sales = df_treatment_sample[df_treatment_sample['converted'] == 1]['converted'].sum()

# Quantidade de Visitas
visit = len(df_treatment_sample)

# Taxa de Conversão do Grupo de Controle
taxa_conversao_tratamento = round(sales/visit, 4)

# ------------ Exibição das Métricas ----------------------

print('Taxa de conversão no grupo de controle: {}%.'.format(taxa_conversao_controle*100))
print('Taxa de conversão no grupo de tratamento: {}%.'.format(taxa_conversao_tratamento*100))

Taxa de conversão no grupo de controle: 11.55%.
Taxa de conversão no grupo de tratamento: 12.9%.


# 4.0 - Teste de Hipóteses

## 4.1 - Determinação do Teste

**1.** Qual o tipo dos dados?
    - Dados Binários.
    
**2.** Qual o tamanho da amostra?
    - 4720
    
**3.** Quantos grupos?
    - 2 Grupos
    
**Teste Escolhido:** Chi-Quadrado

## 4.2 - Teste Chi-Quadrado.

In [27]:
# -------- Ajuste dos dados para o teste -----------------------

# Junção das amostras de controle e tratamento.
df_tabela = pd.concat([df_control_sample, df_treatment_sample])

# Dados para a realização do teste.
df_test = df_tabela[['group', 'converted']].groupby('group').agg(converted = ('converted', 'sum'),
                                                                 non_converted = ('converted', 'count'))

# Reajuste dos não convertidos.
df_test['non_converted'] = df_test['non_converted'] - df_test['converted']

# realização do teste
chi2, p_valor, dof, expected = chi2_contingency(df_test)

print('p-valor do teste: {}'.format(p_valor))
print('Nível de Significância do Teste Bicaudal: {}'.format(round(nivel_significancia/2,3)))

if p_valor < (nivel_significancia/2):
    print('Rejeitamos a hipótese nula.')
else:
    print('Falhamos em rejeitar a hipótese nula.')

p-valor do teste: 0.04776195869945799
Nível de Significância do Teste Bicaudal: 0.025
Falhamos em rejeitar a hipótese nula.


# 5.0 - Respondendo as perguntas do CEO.

**1.** A conversão da nova página é realmente melhor do a conversão da página atual?
    
    - A resposta é não. A sugestão a se dar ao time de negócio é que refaçam o design da página novamente com o objetivo de melhorar a taxa de conversão, ou, realizar uma nova reamostragem que tomaria mais tempo para coleta gerando mais custos para a empresa.

**2.** Qual o potencial de número de vendas que a nova página pode trazer?

    - Como o nosso teste nos disse que a página atual e a nova possuem a mesma taxa de conversão, então o potencial de vendas se mantém o mesmo da página antiga.

**3.** Qual o faturamento total na venda do teclado bluetooth através da nova página?
    
    - Como o nosso teste nos disse que a página atual e a nova possuem a mesma taxa de conversão, então o faturamento total de vendas da página nova se mantém o mesmo em relação ao teclado bluetooth.
    